In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from sklearn import datasets, linear_model
from sklearn.feature_extraction import DictVectorizer

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, SVR
from sklearn.model_selection import KFold
import sklearn
import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
import pandas as pd

In [2]:
f = pd.read_csv('/is/prices/mag_join_lab.csv', sep='|')

print(f.shape)
b = pd.read_csv('/is/prices/mag_bh.csv', sep='|')


(15281, 280)


In [3]:
f = f.merge(b, how='inner', on='ticker')
f['obs'] = 1
f = f.sort_values('date', ascending=False)
print(f.shape)

(15281, 282)


In [24]:
j_neut = 0.8
j_pos = 7.0
j_neg = 6.0
i = 10
label_neut = f'label_{i}_{j_neut}'
label_pos = f'label_{i}_{j_pos}'
label_neg = f'label_{i}_{j_neg}'
print('neut: ', f[label_neut][f[label_neut]==0].shape)
print('neg: ', f[label_neg][f[label_neg]==-1].shape)
print('pos: ', f[label_pos][f[label_pos]==1].shape)

neut:  (2153,)
neg:  (1756,)
pos:  (1913,)


In [ ]:
lagi = [1]
j_neuts = [0.2]
j_poss = [2.0]
j_negs = [2.0]


In [ ]:
lagi = [2]
j_neuts = [0.5]
j_poss = [3.0]
j_negs=[3.0]


In [ ]:
lagi = [5]
j_neuts = [0.5]
j_poss = [5.0]
j_negs = [5.0]


In [ ]:
lagi = [10]
j_neuts = [0.8]
j_poss = [7.0]
j_negs = [6.0]


In [38]:
6 * 3 * 41

738

#### Финальный репорт

In [9]:
from sklearn import metrics
field = 'text_reg_stop_stem'
out = pd.DataFrame()

min_dfs = [5, 10, 15, 25, 35, 50] 
max_dfs = [0.5]
ngram_range_maxs = [1]
lagi = [1]
j_neuts = [0.2]
j_poss = [4.0]
j_negs = [4.0]
days = [1]

counter = 0

t = 4
for i in lagi:
    # split
    for ticker in  f.ticker.unique(): #['sber']: #
        print(ticker)

        for j_neut in j_neuts:
            for j_pos in j_poss:
                for j_neg in j_negs:
                    label_neut = f'label_{i}_{j_neut}'
                    label_pos = f'label_{i}_{j_pos}'
                    label_neg = f'label_{i}_{j_neg}'
                    for day in days:
                        try:

                            ps = f[[label_neut, label_pos, label_neg, field]][(f.ticker!=ticker)] 
                            ps = ps.loc[:,~ps.columns.duplicated()]
                            if t == 0:
                                min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                             ps[(ps[label_neg]==-1)].shape[0])

                                ps1 = ps[(ps[label_pos]==1)].sample(frac=1, random_state=41)
                                ps2 = ps[(ps[label_neg]==-1)].sample(frac=1, random_state=42)

                                df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                                ps2[:min_df].rename(columns={label_neg:'sentiment'}), \
                                               ]).sample(frac=1, random_state=44)

                            else:
                                min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                             ps[(ps[label_neut]==0)].shape[0], \
                                             ps[(ps[label_neg]==-1)].shape[0])

                                ps1 = ps[(ps[label_pos]==1)][[label_pos, field]].sample(frac=1, random_state=41)
                                ps2 = ps[(ps[label_neut]==0)][[label_neut, field]].sample(frac=1, random_state=42)
                                ps3 = ps[(ps[label_neg]==-1)][[label_neg, field]].sample(frac=1, random_state=43)

                                df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                                ps2[:min_df].rename(columns={label_neut:'sentiment'}), \
                                                ps3[:min_df].rename(columns={label_neg:'sentiment'}) \
                                               ]).sample(frac=1, random_state=44)


                            x_train = df[field]
                            y_train = df['sentiment'] 

                        # fit
                            for ngram_range_max in ngram_range_maxs:
                                for min_df in min_dfs:
                                    for max_df in max_dfs:

                                        tf_idf = TfidfVectorizer(min_df=min_df, \
                                                                 max_df=max_df, \
                                                                 ngram_range=(1, ngram_range_max))

                                        train_features = tf_idf.fit_transform(x_train)


                                        model = MultinomialNB(alpha=0.1)
                                        model.fit(train_features, y_train)
                                        y_train_pred = model.predict(train_features)
        # В идеале волатильность взять за эти же лаги наверное
                                        df_temp = pd.DataFrame(f[(f.ticker==ticker)]\
                                                               [[field, 'obs', 'date',\
                                                                 f'change_close_{day}', 'bh', 'close']]).\
            reset_index().drop('index', axis=1)

                                        y_pred = model.predict(tf_idf.transform(df_temp[field]))

                                        df_temp['sentiment'] = y_pred

                                        df_temp['neg'] = 0
                                        df_temp['neg'][df_temp.sentiment==-1] = 1

                                        df_temp['pos'] = 0
                                        df_temp['pos'][df_temp.sentiment==1] = 1

                                        df_temp['neut'] = 0
                                        df_temp['neut'][df_temp.sentiment==0] = 1

                                        df_temp = df_temp.drop_duplicates().\
                                        groupby(['date', f'change_close_{day}', 'bh', 'close']).sum().reset_index()

        # Делаем сум роллинг окно для суммирования сентиментов для периодов больше 1 дня 
        # Нужно удостовериться в сортировке должно быть по возрастанию даты

                                        df_temp[f'close_lag_1'] = df_temp.close.shift(periods=1)
                                        #df_temp[f'close_change_lag_1'] = df_temp.close.shift(periods=1)
                                        df_temp = df_temp.sort_values('date', ascending=True)

                                        df_temp['neg'] = df_temp['neg'].rolling(day).sum()
                                        df_temp['pos'] = df_temp['pos'].rolling(day).sum()
                                        df_temp['neut'] = df_temp['neut'].rolling(day).sum()

                                        df_temp.dropna(subset=['neg', 'pos', 'neut', 'close_lag_1'], inplace=True)
                        # просто сумма
                                        df_temp['sum_sentiment'] = 0
                                        df_temp['sum_sentiment'][(df_temp.pos>df_temp.neg)&(df_temp.pos>=df_temp.neut)] = 1
                                        df_temp['sum_sentiment'][(df_temp.pos<df_temp.neg)&(df_temp.neg>=df_temp.neut)] = -1

        # дивергенция c добавлением 
                                        df_temp['diver'] = ((df_temp.pos - df_temp.neg)**2 \
                                                                  / (df_temp.pos + df_temp.neg)**2)**(1/2)

                                        #df_temp.reset_index(level=[f'change_close_{day}', 'date', 'bh'], inplace=True)
                                        df_temp['clear_return'] = df_temp.sum_sentiment * df_temp[f'change_close_{day}']
                                        df_temp['diver_return'] = df_temp.clear_return * df_temp.diver                

                                        res = dict()
                                        res['counter'] = [counter]
                                        res['ticker'] = [ticker]
                                        res['date'] = [day]

                                        res['min_df'] = [min_df]

                                        res['j_neut'] = [j_neut]
                                        res['j_pos'] = [j_pos] 
                                        res['j_neg'] = [j_neg] 

                                        res['train_size'] = [y_train.shape[0]]
                                        res['test_size'] = [y_pred.shape[0]]

                                        res['obs'] = \
                                        [(df_temp.obs[df_temp.clear_return.isna()==False]).sum()]
                                        res['bh'] = df_temp.bh.max()
                                        res['original_return_isna_all'] = \
                                        [(df_temp[f'change_close_{day}']\
                                          [df_temp[f'change_close_{day}'].isna()==False]).sum()]


                                        res['model_return_isna_all'] = \
                                        [(df_temp.clear_return[df_temp.clear_return.isna()==False]).sum()]

                                        res['model_return_isna_short'] = \
                                        [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=1)]).sum()]

                                        res['model_return_isna_long'] = \
                                        [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=-1)]).sum()]


                                        res['model_diver_return_isna_all'] = \
                                        [(df_temp.diver_return[df_temp.diver_return.isna()==False]).sum()]

                                        res['model_diver_return_isna_short'] = \
                                        [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=1)]).sum()]

                                        res['model_diver_return_isna_long'] = \
                                        [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=-1)]).sum()]






                                        res['train_accuracy'] = [metrics.accuracy_score(y_train_pred, y_train)]


                                        res['train_precision_macro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                                , average='macro')]


                                        res['train_precision_micro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                                , average='micro')]


                                        res['train_recall_micro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                          , average='micro')]


                                        res['train_recall_macro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                          , average='macro')]

                                        res['train_f1_micro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                                  , average='micro')]


                                        res['train_f1_macro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                                  , average='macro')]



                                        x = df_temp[['sum_sentiment', 'close_lag_1']]
                                        x.sum_sentiment = x.sum_sentiment.astype(str)

                                        x = pd.get_dummies(x, drop_first=True)
                                        y = df_temp.close
                                        x2 = sm.add_constant(x)
                                        lm = sm.OLS(y, x2)
                                        res_lm = lm.fit()

                                        res['const'] = res_lm.params[0]
                                        res[f'{x.columns[0]}_coef'] = res_lm.params[1]
                                        res[f'{x.columns[1]}_coef'] = res_lm.params[2]
                                        res[f'{x.columns[2]}_coef'] = res_lm.params[3]

                                        res['const_p'] = res_lm.pvalues[0]
                                        res[f'{x.columns[0]}_p'] = res_lm.pvalues[1]
                                        res[f'{x.columns[1]}_p'] = res_lm.pvalues[2]
                                        res[f'{x.columns[2]}_p'] = res_lm.pvalues[3]
                                        res['r2'] = res_lm.rsquared
                                        res['nobs'] = res_lm.nobs
                                        counter += 1
                                        print(counter, end='\r')


                                        res = pd.DataFrame.from_dict(res)

                                        out = out.append(pd.DataFrame(res))
                                        if counter % 100 == 0: 
                                            out.to_excel(f'/is/res/nb/mag_nb_model_ind_final14.xlsx', index=False)

                        except:
                            continue
    

                    


out.to_excel(f'/is/res/nb/mag_nb_model_ind_final14.xlsx', index=False)
out

irao
sber
lnta
gmkn
vtbr
nmtp
mtss
gazp
tcsg
moex
magn
five
nknc
aflt
rosn
nlmk
rual
kbtk
cbom
rosb
afks
mvid
akrn
aptk
mgnt
qiwi
plzl
tgkd
rtkm
yndx
usbn
svav
nvtk
dvec
kmaz
mail
zill
lkoh
tanl
trcn
tnse


,counter,ticker,date,min_df,j_neut,j_pos,j_neg,train_size,test_size,obs,...,const,close_lag_1_coef,sum_sentiment_0_coef,sum_sentiment_1_coef,const_p,close_lag_1_p,sum_sentiment_0_p,sum_sentiment_1_p,r2,nobs
0,0,irao,1,5,0.2,4.0,4.0,1107,397,396,...,0.022088,0.994947,0.020300,-0.012506,0.313966,0.000000e+00,0.380224,0.618302,0.992291,353.0
0,1,irao,1,10,0.2,4.0,4.0,1107,397,396,...,-0.002931,0.994336,0.043742,0.035603,0.894072,0.000000e+00,0.062147,0.176362,0.992309,353.0
0,2,irao,1,15,0.2,4.0,4.0,1107,397,396,...,0.007746,0.995122,0.027500,0.022034,0.722614,0.000000e+00,0.229077,0.391763,0.992264,353.0
0,3,irao,1,25,0.2,4.0,4.0,1107,397,396,...,0.020037,0.994978,0.017419,-0.007812,0.391269,0.000000e+00,0.473127,0.777790,0.992266,353.0
0,4,irao,1,35,0.2,4.0,4.0,1107,397,396,...,0.012793,0.994688,0.024740,0.005310,0.594261,0.000000e+00,0.329785,0.853725,0.992263,353.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,231,tnse,1,10,0.2,4.0,4.0,1155,78,77,...,-1.491830,1.004775,-21.986664,-15.755683,0.985352,1.735154e-26,0.313387,0.482777,0.861143,65.0
0,232,tnse,1,15,0.2,4.0,4.0,1155,78,77,...,-2.621293,1.010197,-29.793071,-37.953098,0.973256,9.924475e-28,0.146204,0.094128,0.866584,65.0
0,233,tnse,1,25,0.2,4.0,4.0,1155,78,77,...,-13.781049,1.016349,-43.524828,-17.980606,0.859760,4.334243e-28,0.035455,0.430838,0.868595,65.0
0,234,tnse,1,35,0.2,4.0,4.0,1155,78,77,...,10.760123,1.000083,-44.222341,-6.674324,0.886895,1.450781e-28,0.025435,0.760993,0.870520,65.0


In [6]:
from sklearn import metrics
field = 'text_reg_stop_stem'
out = pd.DataFrame()

min_dfs = [5, 10, 15, 25, 35, 50] 
max_dfs = [0.5]
ngram_range_maxs = [1]
lagi = [5]
j_neuts = [0.5]
j_poss = [5.0]
j_negs = [5.0]

days = [1]

counter = 0

t = 4
for i in lagi:
    # split
    for ticker in  f.ticker.unique(): #['sber']: #
        print(ticker)

        for j_neut in j_neuts:
            for j_pos in j_poss:
                for j_neg in j_negs:
                    label_neut = f'label_{i}_{j_neut}'
                    label_pos = f'label_{i}_{j_pos}'
                    label_neg = f'label_{i}_{j_neg}'
                    for day in days:
                        try:

                            ps = f[[label_neut, label_pos, label_neg, field]][(f.ticker!=ticker)] 
                            ps = ps.loc[:,~ps.columns.duplicated()]
                            if t == 0:
                                min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                             ps[(ps[label_neg]==-1)].shape[0])

                                ps1 = ps[(ps[label_pos]==1)].sample(frac=1, random_state=41)
                                ps2 = ps[(ps[label_neg]==-1)].sample(frac=1, random_state=42)

                                df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                                ps2[:min_df].rename(columns={label_neg:'sentiment'}), \
                                               ]).sample(frac=1, random_state=44)

                            else:
                                min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                             ps[(ps[label_neut]==0)].shape[0], \
                                             ps[(ps[label_neg]==-1)].shape[0])

                                ps1 = ps[(ps[label_pos]==1)][[label_pos, field]].sample(frac=1, random_state=41)
                                ps2 = ps[(ps[label_neut]==0)][[label_neut, field]].sample(frac=1, random_state=42)
                                ps3 = ps[(ps[label_neg]==-1)][[label_neg, field]].sample(frac=1, random_state=43)

                                df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                                ps2[:min_df].rename(columns={label_neut:'sentiment'}), \
                                                ps3[:min_df].rename(columns={label_neg:'sentiment'}) \
                                               ]).sample(frac=1, random_state=44)


                            x_train = df[field]
                            y_train = df['sentiment'] 

                        # fit
                            for ngram_range_max in ngram_range_maxs:
                                for min_df in min_dfs:
                                    for max_df in max_dfs:

                                        tf_idf = TfidfVectorizer(min_df=min_df, \
                                                                 max_df=max_df, \
                                                                 ngram_range=(1, ngram_range_max))

                                        train_features = tf_idf.fit_transform(x_train)


                                        model = MultinomialNB(alpha=0.1)
                                        model.fit(train_features, y_train)
                                        y_train_pred = model.predict(train_features)
        # В идеале волатильность взять за эти же лаги наверное
                                        df_temp = pd.DataFrame(f[(f.ticker==ticker)]\
                                                               [[field, 'obs', 'date',\
                                                                 f'change_close_{day}', 'bh', 'close']]).\
            reset_index().drop('index', axis=1)

                                        y_pred = model.predict(tf_idf.transform(df_temp[field]))

                                        df_temp['sentiment'] = y_pred

                                        df_temp['neg'] = 0
                                        df_temp['neg'][df_temp.sentiment==-1] = 1

                                        df_temp['pos'] = 0
                                        df_temp['pos'][df_temp.sentiment==1] = 1

                                        df_temp['neut'] = 0
                                        df_temp['neut'][df_temp.sentiment==0] = 1

                                        df_temp = df_temp.drop_duplicates().\
                                        groupby(['date', f'change_close_{day}', 'bh', 'close']).sum().reset_index()

        # Делаем сум роллинг окно для суммирования сентиментов для периодов больше 1 дня 
        # Нужно удостовериться в сортировке должно быть по возрастанию даты

                                        df_temp[f'close_lag_1'] = df_temp.close.shift(periods=1)
                                        #df_temp[f'close_change_lag_1'] = df_temp.close.shift(periods=1)
                                        df_temp = df_temp.sort_values('date', ascending=True)

                                        df_temp['neg'] = df_temp['neg'].rolling(day).sum()
                                        df_temp['pos'] = df_temp['pos'].rolling(day).sum()
                                        df_temp['neut'] = df_temp['neut'].rolling(day).sum()

                                        df_temp.dropna(subset=['neg', 'pos', 'neut', 'close_lag_1'], inplace=True)
                        # просто сумма
                                        df_temp['sum_sentiment'] = 0
                                        df_temp['sum_sentiment'][(df_temp.pos>df_temp.neg)&(df_temp.pos>=df_temp.neut)] = 1
                                        df_temp['sum_sentiment'][(df_temp.pos<df_temp.neg)&(df_temp.neg>=df_temp.neut)] = -1

        # дивергенция c добавлением 
                                        df_temp['diver'] = ((df_temp.pos - df_temp.neg)**2 \
                                                                  / (df_temp.pos + df_temp.neg)**2)**(1/2)

                                        #df_temp.reset_index(level=[f'change_close_{day}', 'date', 'bh'], inplace=True)
                                        df_temp['clear_return'] = df_temp.sum_sentiment * df_temp[f'change_close_{day}']
                                        df_temp['diver_return'] = df_temp.clear_return * df_temp.diver                

                                        res = dict()
                                        res['counter'] = [counter]
                                        res['ticker'] = [ticker]
                                        res['date'] = [day]

                                        res['min_df'] = [min_df]

                                        res['j_neut'] = [j_neut]
                                        res['j_pos'] = [j_pos] 
                                        res['j_neg'] = [j_neg] 

                                        res['train_size'] = [y_train.shape[0]]
                                        res['test_size'] = [y_pred.shape[0]]

                                        res['obs'] = \
                                        [(df_temp.obs[df_temp.clear_return.isna()==False]).sum()]
                                        res['bh'] = df_temp.bh.max()
                                        res['original_return_isna_all'] = \
                                        [(df_temp[f'change_close_{day}']\
                                          [df_temp[f'change_close_{day}'].isna()==False]).sum()]


                                        res['model_return_isna_all'] = \
                                        [(df_temp.clear_return[df_temp.clear_return.isna()==False]).sum()]

                                        res['model_return_isna_short'] = \
                                        [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=1)]).sum()]

                                        res['model_return_isna_long'] = \
                                        [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=-1)]).sum()]


                                        res['model_diver_return_isna_all'] = \
                                        [(df_temp.diver_return[df_temp.diver_return.isna()==False]).sum()]

                                        res['model_diver_return_isna_short'] = \
                                        [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=1)]).sum()]

                                        res['model_diver_return_isna_long'] = \
                                        [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=-1)]).sum()]






                                        res['train_accuracy'] = [metrics.accuracy_score(y_train_pred, y_train)]


                                        res['train_precision_macro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                                , average='macro')]


                                        res['train_precision_micro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                                , average='micro')]


                                        res['train_recall_micro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                          , average='micro')]


                                        res['train_recall_macro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                          , average='macro')]

                                        res['train_f1_micro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                                  , average='micro')]


                                        res['train_f1_macro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                                  , average='macro')]



                                        x = df_temp[['sum_sentiment', 'close_lag_1']]
                                        x.sum_sentiment = x.sum_sentiment.astype(str)

                                        x = pd.get_dummies(x, drop_first=True)
                                        y = df_temp.close
                                        x2 = sm.add_constant(x)
                                        lm = sm.OLS(y, x2)
                                        res_lm = lm.fit()

                                        res['const'] = res_lm.params[0]
                                        res[f'{x.columns[0]}_coef'] = res_lm.params[1]
                                        res[f'{x.columns[1]}_coef'] = res_lm.params[2]
                                        res[f'{x.columns[2]}_coef'] = res_lm.params[3]

                                        res['const_p'] = res_lm.pvalues[0]
                                        res[f'{x.columns[0]}_p'] = res_lm.pvalues[1]
                                        res[f'{x.columns[1]}_p'] = res_lm.pvalues[2]
                                        res[f'{x.columns[2]}_p'] = res_lm.pvalues[3]
                                        res['r2'] = res_lm.rsquared
                                        res['nobs'] = res_lm.nobs
                                        counter += 1
                                        print(counter, end='\r')


                                        res = pd.DataFrame.from_dict(res)

                                        out = out.append(pd.DataFrame(res))
                                        if counter % 100 == 0: 
                                            out.to_excel(f'/is/res/nb/mag_nb_model_ind_final5.xlsx', index=False)

                        except:
                            continue
    

                    


out.to_excel(f'/is/res/nb/mag_nb_model_ind_final5.xlsx', index=False)
out

irao
sber
lnta
gmkn
vtbr
nmtp
mtss
gazp
tcsg
moex
magn
five
nknc
aflt
rosn
nlmk
rual
kbtk
cbom
rosb
afks
mvid
akrn
aptk
mgnt
qiwi
plzl
tgkd
rtkm
yndx
usbn
svav
nvtk
dvec
kmaz
mail
zill
lkoh
tanl
trcn
tnse


,counter,ticker,date,min_df,j_neut,j_pos,j_neg,train_size,test_size,obs,...,const,close_lag_1_coef,sum_sentiment_0_coef,sum_sentiment_1_coef,const_p,close_lag_1_p,sum_sentiment_0_p,sum_sentiment_1_p,r2,nobs
0,0,irao,1,5,0.5,5.0,5.0,3951,397,396,...,-0.002187,0.995395,0.036158,0.041094,0.919905,0.000000e+00,0.107166,0.100776,0.992304,353.0
0,1,irao,1,10,0.5,5.0,5.0,3951,397,396,...,-0.003152,0.995284,0.037770,0.039181,0.889395,0.000000e+00,0.104901,0.130059,0.992298,353.0
0,2,irao,1,15,0.5,5.0,5.0,3951,397,396,...,-0.003575,0.994988,0.040802,0.039799,0.870592,0.000000e+00,0.072995,0.121610,0.992309,353.0
0,3,irao,1,25,0.5,5.0,5.0,3951,397,396,...,0.000168,0.995195,0.032905,0.042506,0.993658,0.000000e+00,0.138009,0.094869,0.992302,353.0
0,4,irao,1,35,0.5,5.0,5.0,3951,397,396,...,-0.001384,0.995418,0.033274,0.051978,0.946344,0.000000e+00,0.116931,0.046646,0.992326,353.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,230,tnse,1,10,0.5,5.0,5.0,4059,78,77,...,16.574552,0.994710,-46.694706,3.683718,0.823382,6.504384e-29,0.051590,0.867194,0.874968,65.0
0,231,tnse,1,15,0.5,5.0,5.0,4059,78,77,...,38.657930,0.989769,-62.374937,-17.280577,0.599894,6.724603e-29,0.006494,0.416565,0.877552,65.0
0,232,tnse,1,25,0.5,5.0,5.0,4059,78,77,...,14.791721,1.007732,-59.528148,-24.810013,0.843828,1.008011e-28,0.018685,0.282102,0.872235,65.0
0,233,tnse,1,35,0.5,5.0,5.0,4059,78,77,...,22.701828,0.994609,-34.993315,-13.381386,0.770505,7.173716e-28,0.187765,0.597589,0.863572,65.0


In [7]:
from sklearn import metrics
field = 'text_reg_stop_stem'
out = pd.DataFrame()

min_dfs = [5, 10, 15, 25, 35, 50] 
max_dfs = [0.5]
ngram_range_maxs = [1]
lagi = [2]
j_neuts = [0.5]
j_poss = [3.0]
j_negs=[3.0]
days = [1]

counter = 0

t = 4
for i in lagi:
    # split
    for ticker in  f.ticker.unique(): #['sber']: #
        print(ticker)

        for j_neut in j_neuts:
            for j_pos in j_poss:
                for j_neg in j_negs:
                    label_neut = f'label_{i}_{j_neut}'
                    label_pos = f'label_{i}_{j_pos}'
                    label_neg = f'label_{i}_{j_neg}'
                    for day in days:
                        try:

                            ps = f[[label_neut, label_pos, label_neg, field]][(f.ticker!=ticker)] 
                            ps = ps.loc[:,~ps.columns.duplicated()]
                            if t == 0:
                                min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                             ps[(ps[label_neg]==-1)].shape[0])

                                ps1 = ps[(ps[label_pos]==1)].sample(frac=1, random_state=41)
                                ps2 = ps[(ps[label_neg]==-1)].sample(frac=1, random_state=42)

                                df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                                ps2[:min_df].rename(columns={label_neg:'sentiment'}), \
                                               ]).sample(frac=1, random_state=44)

                            else:
                                min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                             ps[(ps[label_neut]==0)].shape[0], \
                                             ps[(ps[label_neg]==-1)].shape[0])

                                ps1 = ps[(ps[label_pos]==1)][[label_pos, field]].sample(frac=1, random_state=41)
                                ps2 = ps[(ps[label_neut]==0)][[label_neut, field]].sample(frac=1, random_state=42)
                                ps3 = ps[(ps[label_neg]==-1)][[label_neg, field]].sample(frac=1, random_state=43)

                                df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                                ps2[:min_df].rename(columns={label_neut:'sentiment'}), \
                                                ps3[:min_df].rename(columns={label_neg:'sentiment'}) \
                                               ]).sample(frac=1, random_state=44)


                            x_train = df[field]
                            y_train = df['sentiment'] 

                        # fit
                            for ngram_range_max in ngram_range_maxs:
                                for min_df in min_dfs:
                                    for max_df in max_dfs:

                                        tf_idf = TfidfVectorizer(min_df=min_df, \
                                                                 max_df=max_df, \
                                                                 ngram_range=(1, ngram_range_max))

                                        train_features = tf_idf.fit_transform(x_train)


                                        model = MultinomialNB(alpha=0.1)
                                        model.fit(train_features, y_train)
                                        y_train_pred = model.predict(train_features)
        # В идеале волатильность взять за эти же лаги наверное
                                        df_temp = pd.DataFrame(f[(f.ticker==ticker)]\
                                                               [[field, 'obs', 'date',\
                                                                 f'change_close_{day}', 'bh', 'close']]).\
            reset_index().drop('index', axis=1)

                                        y_pred = model.predict(tf_idf.transform(df_temp[field]))

                                        df_temp['sentiment'] = y_pred

                                        df_temp['neg'] = 0
                                        df_temp['neg'][df_temp.sentiment==-1] = 1

                                        df_temp['pos'] = 0
                                        df_temp['pos'][df_temp.sentiment==1] = 1

                                        df_temp['neut'] = 0
                                        df_temp['neut'][df_temp.sentiment==0] = 1

                                        df_temp = df_temp.drop_duplicates().\
                                        groupby(['date', f'change_close_{day}', 'bh', 'close']).sum().reset_index()

        # Делаем сум роллинг окно для суммирования сентиментов для периодов больше 1 дня 
        # Нужно удостовериться в сортировке должно быть по возрастанию даты

                                        df_temp[f'close_lag_1'] = df_temp.close.shift(periods=1)
                                        #df_temp[f'close_change_lag_1'] = df_temp.close.shift(periods=1)
                                        df_temp = df_temp.sort_values('date', ascending=True)

                                        df_temp['neg'] = df_temp['neg'].rolling(day).sum()
                                        df_temp['pos'] = df_temp['pos'].rolling(day).sum()
                                        df_temp['neut'] = df_temp['neut'].rolling(day).sum()

                                        df_temp.dropna(subset=['neg', 'pos', 'neut', 'close_lag_1'], inplace=True)
                        # просто сумма
                                        df_temp['sum_sentiment'] = 0
                                        df_temp['sum_sentiment'][(df_temp.pos>df_temp.neg)&(df_temp.pos>=df_temp.neut)] = 1
                                        df_temp['sum_sentiment'][(df_temp.pos<df_temp.neg)&(df_temp.neg>=df_temp.neut)] = -1

        # дивергенция c добавлением 
                                        df_temp['diver'] = ((df_temp.pos - df_temp.neg)**2 \
                                                                  / (df_temp.pos + df_temp.neg)**2)**(1/2)

                                        #df_temp.reset_index(level=[f'change_close_{day}', 'date', 'bh'], inplace=True)
                                        df_temp['clear_return'] = df_temp.sum_sentiment * df_temp[f'change_close_{day}']
                                        df_temp['diver_return'] = df_temp.clear_return * df_temp.diver                

                                        res = dict()
                                        res['counter'] = [counter]
                                        res['ticker'] = [ticker]
                                        res['date'] = [day]

                                        res['min_df'] = [min_df]

                                        res['j_neut'] = [j_neut]
                                        res['j_pos'] = [j_pos] 
                                        res['j_neg'] = [j_neg] 

                                        res['train_size'] = [y_train.shape[0]]
                                        res['test_size'] = [y_pred.shape[0]]

                                        res['obs'] = \
                                        [(df_temp.obs[df_temp.clear_return.isna()==False]).sum()]
                                        res['bh'] = df_temp.bh.max()
                                        res['original_return_isna_all'] = \
                                        [(df_temp[f'change_close_{day}']\
                                          [df_temp[f'change_close_{day}'].isna()==False]).sum()]


                                        res['model_return_isna_all'] = \
                                        [(df_temp.clear_return[df_temp.clear_return.isna()==False]).sum()]

                                        res['model_return_isna_short'] = \
                                        [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=1)]).sum()]

                                        res['model_return_isna_long'] = \
                                        [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=-1)]).sum()]


                                        res['model_diver_return_isna_all'] = \
                                        [(df_temp.diver_return[df_temp.diver_return.isna()==False]).sum()]

                                        res['model_diver_return_isna_short'] = \
                                        [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=1)]).sum()]

                                        res['model_diver_return_isna_long'] = \
                                        [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=-1)]).sum()]






                                        res['train_accuracy'] = [metrics.accuracy_score(y_train_pred, y_train)]


                                        res['train_precision_macro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                                , average='macro')]


                                        res['train_precision_micro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                                , average='micro')]


                                        res['train_recall_micro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                          , average='micro')]


                                        res['train_recall_macro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                          , average='macro')]

                                        res['train_f1_micro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                                  , average='micro')]


                                        res['train_f1_macro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                                  , average='macro')]



                                        x = df_temp[['sum_sentiment', 'close_lag_1']]
                                        x.sum_sentiment = x.sum_sentiment.astype(str)

                                        x = pd.get_dummies(x, drop_first=True)
                                        y = df_temp.close
                                        x2 = sm.add_constant(x)
                                        lm = sm.OLS(y, x2)
                                        res_lm = lm.fit()

                                        res['const'] = res_lm.params[0]
                                        res[f'{x.columns[0]}_coef'] = res_lm.params[1]
                                        res[f'{x.columns[1]}_coef'] = res_lm.params[2]
                                        res[f'{x.columns[2]}_coef'] = res_lm.params[3]

                                        res['const_p'] = res_lm.pvalues[0]
                                        res[f'{x.columns[0]}_p'] = res_lm.pvalues[1]
                                        res[f'{x.columns[1]}_p'] = res_lm.pvalues[2]
                                        res[f'{x.columns[2]}_p'] = res_lm.pvalues[3]
                                        res['r2'] = res_lm.rsquared
                                        res['nobs'] = res_lm.nobs
                                        counter += 1
                                        print(counter, end='\r')


                                        res = pd.DataFrame.from_dict(res)

                                        out = out.append(pd.DataFrame(res))
                                        if counter % 100 == 0: 
                                            out.to_excel(f'/is/res/nb/mag_nb_model_ind_final2.xlsx', index=False)

                        except:
                            continue
    

                    


out.to_excel(f'/is/res/nb/mag_nb_model_ind_final2.xlsx', index=False)
out

irao
sber
lnta
gmkn
vtbr
nmtp
mtss
gazp
tcsg
moex
magn
five
nknc
aflt
rosn
nlmk
rual
kbtk
cbom
rosb
afks
mvid
akrn
aptk
mgnt
qiwi
plzl
tgkd
rtkm
yndx
usbn
svav
nvtk
dvec
kmaz
mail
zill
lkoh
tanl
trcn
tnse


,counter,ticker,date,min_df,j_neut,j_pos,j_neg,train_size,test_size,obs,...,const,close_lag_1_coef,sum_sentiment_0_coef,sum_sentiment_1_coef,const_p,close_lag_1_p,sum_sentiment_0_p,sum_sentiment_1_p,r2,nobs
0,0,irao,1,5,0.5,3.0,3.0,4428,397,396,...,0.013352,0.995037,0.033523,-0.002810,0.454957,0.000000e+00,0.083804,0.914287,0.992316,353.0
0,1,irao,1,10,0.5,3.0,3.0,4428,397,396,...,0.007695,0.995297,0.037167,0.004607,0.684458,0.000000e+00,0.058652,0.863650,0.992324,353.0
0,2,irao,1,15,0.5,3.0,3.0,4428,397,396,...,0.016416,0.995494,0.018855,0.002992,0.391520,0.000000e+00,0.342278,0.914357,0.992255,353.0
0,3,irao,1,25,0.5,3.0,3.0,4428,397,396,...,0.017133,0.995899,0.025055,-0.026337,0.398162,0.000000e+00,0.230707,0.320270,0.992347,353.0
0,4,irao,1,35,0.5,3.0,3.0,4428,397,396,...,0.013274,0.995389,0.028682,-0.011929,0.511870,0.000000e+00,0.184043,0.660804,0.992317,353.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,238,tnse,1,10,0.5,3.0,3.0,4557,78,77,...,30.699063,0.998136,-45.932711,-24.457205,0.687607,3.171050e-28,0.028274,0.420336,0.869777,65.0
0,239,tnse,1,15,0.5,3.0,3.0,4557,78,77,...,22.376036,1.005592,-46.658071,-28.104598,0.770727,7.579096e-28,0.043368,0.384024,0.868196,65.0
0,240,tnse,1,25,0.5,3.0,3.0,4557,78,77,...,36.119558,1.004621,-63.087892,-34.542620,0.626290,5.033891e-29,0.006661,0.241557,0.875859,65.0
0,241,tnse,1,35,0.5,3.0,3.0,4557,78,77,...,23.745466,0.998510,-39.895058,-12.685262,0.755626,3.384539e-28,0.054391,0.658749,0.867953,65.0


In [8]:
from sklearn import metrics
field = 'text_reg_stop_stem'
out = pd.DataFrame()

min_dfs = [5, 10, 15, 25, 35, 50] 
max_dfs = [0.5]
ngram_range_maxs = [1]

lagi = [1]
j_neuts = [0.2]
j_poss = [2.0]
j_negs = [2.0]

days = [1]

counter = 0

t = 4
for i in lagi:
    # split
    for ticker in  f.ticker.unique(): #['sber']: #
        print(ticker)

        for j_neut in j_neuts:
            for j_pos in j_poss:
                for j_neg in j_negs:
                    label_neut = f'label_{i}_{j_neut}'
                    label_pos = f'label_{i}_{j_pos}'
                    label_neg = f'label_{i}_{j_neg}'
                    for day in days:
                        try:

                            ps = f[[label_neut, label_pos, label_neg, field]][(f.ticker!=ticker)] 
                            ps = ps.loc[:,~ps.columns.duplicated()]
                            if t == 0:
                                min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                             ps[(ps[label_neg]==-1)].shape[0])

                                ps1 = ps[(ps[label_pos]==1)].sample(frac=1, random_state=41)
                                ps2 = ps[(ps[label_neg]==-1)].sample(frac=1, random_state=42)

                                df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                                ps2[:min_df].rename(columns={label_neg:'sentiment'}), \
                                               ]).sample(frac=1, random_state=44)

                            else:
                                min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                             ps[(ps[label_neut]==0)].shape[0], \
                                             ps[(ps[label_neg]==-1)].shape[0])

                                ps1 = ps[(ps[label_pos]==1)][[label_pos, field]].sample(frac=1, random_state=41)
                                ps2 = ps[(ps[label_neut]==0)][[label_neut, field]].sample(frac=1, random_state=42)
                                ps3 = ps[(ps[label_neg]==-1)][[label_neg, field]].sample(frac=1, random_state=43)

                                df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                                ps2[:min_df].rename(columns={label_neut:'sentiment'}), \
                                                ps3[:min_df].rename(columns={label_neg:'sentiment'}) \
                                               ]).sample(frac=1, random_state=44)


                            x_train = df[field]
                            y_train = df['sentiment'] 

                        # fit
                            for ngram_range_max in ngram_range_maxs:
                                for min_df in min_dfs:
                                    for max_df in max_dfs:

                                        tf_idf = TfidfVectorizer(min_df=min_df, \
                                                                 max_df=max_df, \
                                                                 ngram_range=(1, ngram_range_max))

                                        train_features = tf_idf.fit_transform(x_train)


                                        model = MultinomialNB(alpha=0.1)
                                        model.fit(train_features, y_train)
                                        y_train_pred = model.predict(train_features)
        # В идеале волатильность взять за эти же лаги наверное
                                        df_temp = pd.DataFrame(f[(f.ticker==ticker)]\
                                                               [[field, 'obs', 'date',\
                                                                 f'change_close_{day}', 'bh', 'close']]).\
            reset_index().drop('index', axis=1)

                                        y_pred = model.predict(tf_idf.transform(df_temp[field]))

                                        df_temp['sentiment'] = y_pred

                                        df_temp['neg'] = 0
                                        df_temp['neg'][df_temp.sentiment==-1] = 1

                                        df_temp['pos'] = 0
                                        df_temp['pos'][df_temp.sentiment==1] = 1

                                        df_temp['neut'] = 0
                                        df_temp['neut'][df_temp.sentiment==0] = 1

                                        df_temp = df_temp.drop_duplicates().\
                                        groupby(['date', f'change_close_{day}', 'bh', 'close']).sum().reset_index()

        # Делаем сум роллинг окно для суммирования сентиментов для периодов больше 1 дня 
        # Нужно удостовериться в сортировке должно быть по возрастанию даты

                                        df_temp[f'close_lag_1'] = df_temp.close.shift(periods=1)
                                        #df_temp[f'close_change_lag_1'] = df_temp.close.shift(periods=1)
                                        df_temp = df_temp.sort_values('date', ascending=True)

                                        df_temp['neg'] = df_temp['neg'].rolling(day).sum()
                                        df_temp['pos'] = df_temp['pos'].rolling(day).sum()
                                        df_temp['neut'] = df_temp['neut'].rolling(day).sum()

                                        df_temp.dropna(subset=['neg', 'pos', 'neut', 'close_lag_1'], inplace=True)
                        # просто сумма
                                        df_temp['sum_sentiment'] = 0
                                        df_temp['sum_sentiment'][(df_temp.pos>df_temp.neg)&(df_temp.pos>=df_temp.neut)] = 1
                                        df_temp['sum_sentiment'][(df_temp.pos<df_temp.neg)&(df_temp.neg>=df_temp.neut)] = -1

        # дивергенция c добавлением 
                                        df_temp['diver'] = ((df_temp.pos - df_temp.neg)**2 \
                                                                  / (df_temp.pos + df_temp.neg)**2)**(1/2)

                                        #df_temp.reset_index(level=[f'change_close_{day}', 'date', 'bh'], inplace=True)
                                        df_temp['clear_return'] = df_temp.sum_sentiment * df_temp[f'change_close_{day}']
                                        df_temp['diver_return'] = df_temp.clear_return * df_temp.diver                

                                        res = dict()
                                        res['counter'] = [counter]
                                        res['ticker'] = [ticker]
                                        res['date'] = [day]

                                        res['min_df'] = [min_df]

                                        res['j_neut'] = [j_neut]
                                        res['j_pos'] = [j_pos] 
                                        res['j_neg'] = [j_neg] 

                                        res['train_size'] = [y_train.shape[0]]
                                        res['test_size'] = [y_pred.shape[0]]

                                        res['obs'] = \
                                        [(df_temp.obs[df_temp.clear_return.isna()==False]).sum()]
                                        res['bh'] = df_temp.bh.max()
                                        res['original_return_isna_all'] = \
                                        [(df_temp[f'change_close_{day}']\
                                          [df_temp[f'change_close_{day}'].isna()==False]).sum()]


                                        res['model_return_isna_all'] = \
                                        [(df_temp.clear_return[df_temp.clear_return.isna()==False]).sum()]

                                        res['model_return_isna_short'] = \
                                        [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=1)]).sum()]

                                        res['model_return_isna_long'] = \
                                        [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=-1)]).sum()]


                                        res['model_diver_return_isna_all'] = \
                                        [(df_temp.diver_return[df_temp.diver_return.isna()==False]).sum()]

                                        res['model_diver_return_isna_short'] = \
                                        [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=1)]).sum()]

                                        res['model_diver_return_isna_long'] = \
                                        [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                     (df_temp.sum_sentiment!=-1)]).sum()]






                                        res['train_accuracy'] = [metrics.accuracy_score(y_train_pred, y_train)]


                                        res['train_precision_macro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                                , average='macro')]


                                        res['train_precision_micro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                                , average='micro')]


                                        res['train_recall_micro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                          , average='micro')]


                                        res['train_recall_macro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                          , average='macro')]

                                        res['train_f1_micro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                                  , average='micro')]


                                        res['train_f1_macro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                                  , average='macro')]



                                        x = df_temp[['sum_sentiment', 'close_lag_1']]
                                        x.sum_sentiment = x.sum_sentiment.astype(str)

                                        x = pd.get_dummies(x, drop_first=True)
                                        y = df_temp.close
                                        x2 = sm.add_constant(x)
                                        lm = sm.OLS(y, x2)
                                        res_lm = lm.fit()

                                        res['const'] = res_lm.params[0]
                                        res[f'{x.columns[0]}_coef'] = res_lm.params[1]
                                        res[f'{x.columns[1]}_coef'] = res_lm.params[2]
                                        res[f'{x.columns[2]}_coef'] = res_lm.params[3]

                                        res['const_p'] = res_lm.pvalues[0]
                                        res[f'{x.columns[0]}_p'] = res_lm.pvalues[1]
                                        res[f'{x.columns[1]}_p'] = res_lm.pvalues[2]
                                        res[f'{x.columns[2]}_p'] = res_lm.pvalues[3]
                                        res['r2'] = res_lm.rsquared
                                        res['nobs'] = res_lm.nobs
                                        counter += 1
                                        print(counter, end='\r')


                                        res = pd.DataFrame.from_dict(res)

                                        out = out.append(pd.DataFrame(res))
                                        if counter % 100 == 0: 
                                            out.to_excel(f'/is/res/nb/mag_nb_model_ind_final1.xlsx', index=False)

                        except:
                            continue
    

                    


out.to_excel(f'/is/res/nb/mag_nb_model_ind_final1.xlsx', index=False)
out

irao
sber
lnta
gmkn
vtbr
nmtp
mtss
gazp
tcsg
moex
magn
five
nknc
aflt
rosn
nlmk
rual
kbtk
cbom
rosb
afks
mvid
akrn
aptk
mgnt
qiwi
plzl
tgkd
rtkm
yndx
usbn
svav
nvtk
dvec
kmaz
mail
zill
lkoh
tanl
trcn
tnse


,counter,ticker,date,min_df,j_neut,j_pos,j_neg,train_size,test_size,obs,...,const,close_lag_1_coef,sum_sentiment_0_coef,sum_sentiment_1_coef,const_p,close_lag_1_p,sum_sentiment_0_p,sum_sentiment_1_p,r2,nobs
0,0,irao,1,5,0.2,2.0,2.0,4647,397,396,...,0.018642,0.994959,0.022678,-0.006244,0.332967,0.000000e+00,0.273654,0.802503,0.992278,353.0
0,1,irao,1,10,0.2,2.0,2.0,4647,397,396,...,0.022373,0.995269,0.021382,-0.026709,0.246910,0.000000e+00,0.292939,0.288758,0.992331,353.0
0,2,irao,1,15,0.2,2.0,2.0,4647,397,396,...,0.027999,0.995567,0.010095,-0.034554,0.165103,0.000000e+00,0.630488,0.188959,0.992314,353.0
0,3,irao,1,25,0.2,2.0,2.0,4647,397,396,...,0.021841,0.995321,0.013395,-0.013447,0.311424,0.000000e+00,0.549341,0.631555,0.992263,353.0
0,4,irao,1,35,0.2,2.0,2.0,4647,397,396,...,0.047335,0.996576,-0.021679,-0.053987,0.029519,0.000000e+00,0.332167,0.052995,0.992316,353.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,234,tnse,1,10,0.2,2.0,2.0,4791,78,77,...,23.100664,0.987808,-28.066861,29.452588,0.764141,1.094457e-28,0.122286,0.261891,0.871237,65.0
0,235,tnse,1,15,0.2,2.0,2.0,4791,78,77,...,15.211824,0.991373,-26.111049,33.703271,0.839886,6.889696e-29,0.152358,0.179464,0.872216,65.0
0,236,tnse,1,25,0.2,2.0,2.0,4791,78,77,...,22.680179,0.994064,-44.057548,-14.364444,0.764399,1.128744e-28,0.026233,0.500154,0.869851,65.0
0,237,tnse,1,35,0.2,2.0,2.0,4791,78,77,...,15.265632,0.999254,-47.911913,-14.159434,0.839356,6.937787e-29,0.016183,0.499672,0.871696,65.0
